In [1]:
## Finds posterior distribution of observed values assuming that k follows a binomial distribution. p is discrete in this case.

numerator <- function(k, n, posprobs, probsofprobs){
  ## finds the numerator -- ie P(k|p)P(p)
  ## k (integer) : the total number of observed successes.
  ## n (integer) : the total number of observed trials.
  ## posprobs (vector): possible p values associated with the random variable k.
  ## probsofprobs (vector): this is our prior, mapping a probability to each possible probability

  function(p){
    ## For a particular p, this will find the posterior probability of p given observed values. This is a function generator -- given
    ## values of k and n, we can then find the probability of any particular p.
    x <- posprobs[p]
    px <- probsofprobs[p]

    res <- (x^k)*((1 - x)^(n-k))*px
    return(res)
  }
}

denominator <- function(k,n, posprobs, probsofprobs){
  ## Finds the denominator, the multiplicative constant. Same for all values of p. P(k)
  sum(sapply(X = 1:length(posprobs),FUN= numerator(k,n, posprobs, probsofprobs)))
}

probabilityfinder <- function(k,n,p, posprobs, probsofprobs){
  ## Finds the posterior probability of a particular p value.
  numerator(k,n, posprobs, probsofprobs)(p)/denominator(k,n, posprobs, probsofprobs)
}

posterior <- function(k,n, posprobs, probsofprobs){
  ## Finds the posterior probability mass function of the p values.
  sapply(X = 1:length(posprobs),FUN=function(p) probabilityfinder(k,n,p, posprobs, probsofprobs))
}

log_transf_numerator <- function(k,n){
  ## finds the numerator -- ie P(k|p)P(p), using log transform to avoid machine zeros
  ## k (integer) : the total number of observed successes.
  ## n (integer) : the total number of observed trials.
  ## posprobs (vector): possible p values associated with the random variable k.
  ## probsofprobs (vector): this is our prior, mapping a probability to each possible probability
  function(p){
    ## For a particular p, this will find the posterior probability of p given observed values. This is a function generator -- given
    ## values of k and n, we can then find the probability of any particular p.

    x <- posprobs[p]
    px <- probsofprobs[p]

    mx <- max(sapply(posprobs, function(pr) k*log(pr) + (n - k)*log(1 - pr)))

    res <- exp(k*log(x) + (n - k)*log(1 - x) - mx) * probsofprobs[p]
    return(res)
  }
}

log_transf_denominator <- function(k,n){
  ## Finds the denominator, the multiplicative constant, using log transform for machine zeros. Same for all values of p. P(k)

  sum(sapply(X = 1:length(posprobs),FUN= log_transf_numerator(k,n)))
}

log_transf_probabilitydist <- function(k,n,p){
  ## Finds the posterior probability of a particular p value using log transform.
  log_transf_numerator(k,n)(p)/log_transf_denominator(k,n)
}

log_transf_posterior <- function(k,n){
  ## Finds the posterior probability mass function of the p values using log transform

  sapply(X = 1:length(posprobs),FUN=function(x) log_transf_probabilitydist(k,n,x))
}



# Example -- Thunderstorms
Example: consider if we observed 11 events -- as an example, the number of thunderstorms in a month. We want to estimate the probability of a thunderstorm
given the new data. Assuming weather each day is independent (which is a simplifying assumption), we can assume that the number of thunderstorms in n days can be modeled by
a binomial distribution. If we believe prior to this that the distribution looks as it does below (with the highest belief in 35 and 45%), we can now find the
posterior of the probabilities after observing the 11 thunderstorms as follows:


In [2]:
posprobs <- c(.05,.15,.25,.35,.45,.55,.65,.75,.85,.95)
probsofprobs <- c(.07,.13,.26,.26,.13,.07,.02,.02,.02,.02)
length(posprobs) == length(probsofprobs)

pos <- posterior(11,27,posprobs, probsofprobs)
pos

[1] TRUE

[1] 3.095425e-08 1.718005e-03 1.278404e-01 5.244571e-01 2.873813e-01
 [6] 5.673647e-02 1.826165e-03 4.046863e-05 4.523481e-08 3.571776e-15

So we now believe with `r round(pos[which.max(pos)],2)`  probability that the actual p value is `r posprobs[which.max(pos)]`. This means the data supports the assumption of `r posprobs[which.max(pos)]` chance of thunderstorms a day, given our prior beliefs and the independence of events.

# Behaviors for different n and k.



In [3]:
n <- c(10,50,100,150,200,300,500)
k <- lapply(n, function(n) seq(n/10,n,n/10))
lapply(1:7,function(i) sapply(1:10, function(j) posterior(k[[i]][j],n[i], posprobs, probsofprobs)))

1.599929e-01,3.044837e-02,4.175353e-03,4.535141e-04,3.970198e-05,0.0000027423,1.387873e-07,4.323270e-09,6.308829e-11,3.898065e-13
3.275858e-01,2.090328e-01,9.611020e-02,3.500202e-02,1.027402e-02,0.0023794092,4.037659e-04,4.217143e-05,2.063387e-06,4.274713e-08
3.539850e-01,4.266587e-01,3.705460e-01,2.549013e-01,1.413272e-01,0.0618245522,1.981654e-02,3.909516e-03,3.613198e-04,1.413919e-05
1.367042e-01,2.661665e-01,3.734142e-01,4.149510e-01,3.716434e-01,0.2626259078,1.359817e-01,4.333627e-02,6.469879e-03,4.089826e-04
1.954070e-02,5.781053e-02,1.232366e-01,2.080849e-01,2.831819e-01,0.3040687011,2.392268e-01,1.158446e-01,2.627942e-02,2.524175e-03
2.112971e-03,9.338145e-03,2.973675e-02,7.500583e-02,1.524825e-01,0.2445832762,2.874519e-01,2.079370e-01,7.046474e-02,1.011058e-02
7.431490e-05,4.990432e-04,2.414717e-03,9.254710e-03,2.858795e-02,0.0696762613,1.244281e-01,1.367666e-01,7.042313e-02,1.535375e-02
4.150223e-06,4.502043e-05,3.518955e-04,2.178644e-03,1.087132e-02,0.0428016131,1.234723e-01,2.192337e-01,1.823553e-01,6.422345e-02
4.740131e-08,9.712589e-07,1.433988e-05,1.676966e-04,1.580618e-03,0.0117546882,6.405116e-02,2.148182e-01,3.375116e-01,2.245279e-01
2.691558e-12,1.849159e-10,9.153996e-09,3.589348e-07,1.134342e-05,0.0002828487,5.167684e-03,5.811202e-02,3.061326e-01,6.828370e-01
2.322306e-01,2.277038e-04,1.627936e-08,1.818866e-13,3.574528e-19,1.133380e-25,4.692851e-33,9.915429e-42,9.158596e-52,4.024562e-64


Each list represents n = 10,50,100,150,200,300,500, and each column represents possible ks from n/10 to n.
We see what is expected with Bayesian probabilities -- as n increases, our confidence in probabilities becomes stronger -- we get values that are nearly 1 for the observations
which have p = k/n and 0 otherwise. This shows that the prior does not matter in these cases, as we get more observations, the data will overwhelm the prior.
For small values of n, we see the opposite -- it is highly skewed by our prior, so that we are not so confident that p = k/n if p was not deemed probable prior.

# Machine Zeros problem:


In [4]:
posterior(1000,2000, posprobs, probsofprobs)

[1] NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN

Notice that we have many machine zeros for large n. This interferes with our calculations, and gives us NaNs for the calculations.
The following uses a log transformation to avoid machine zeros:



In [5]:
posterior(1000,2000, posprobs, probsofprobs)
log_transf_posterior(1000,2000)

[1] NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN

[1]  0.000000e+00 5.596221e-289 3.467465e-121  3.312387e-37  6.500000e-01
 [6]  3.500000e-01  2.547990e-38 2.667281e-122 8.609571e-290  0.000000e+00